In [21]:
import telebot
from telebot import *
import requests
import validators

f = open('tgkey.txt', 'r', encoding="utf-8")
bot = telebot.TeleBot(f.read())
f.close()

def sendkeyboard(keyid,message):
    if keyid==0:
        keyboard = types.InlineKeyboardMarkup()
        key_b1 = types.InlineKeyboardButton(text='Доступность сайта', callback_data='b1')
        keyboard.add(key_b1)
        key_b2= types.InlineKeyboardButton(text='Анализ текста', callback_data='b2')
        keyboard.add(key_b2)
        key_b3= types.InlineKeyboardButton(text='Калькулятор', callback_data='b3')
        keyboard.add(key_b3)
        bot.send_message(message.chat.id, text='Чем могу помочь?', reply_markup=keyboard)
    elif keyid==1:
        keyboard = types.InlineKeyboardMarkup()
        key_b4 = types.InlineKeyboardButton(text='*', callback_data='b4')
        key_b5= types.InlineKeyboardButton(text='/', callback_data='b5')
        key_b6= types.InlineKeyboardButton(text='+', callback_data='b6')
        key_b7= types.InlineKeyboardButton(text='-', callback_data='b7')
        keyboard.row(key_b4, key_b5, key_b6, key_b7)
        key_b8= types.InlineKeyboardButton(text='Back', callback_data='b8')
        keyboard.add(key_b8)
        bot.send_message(message.chat.id, text='Выберите операцию', reply_markup=keyboard)

@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    if message.text == '/start':
        sendkeyboard(0, message)
    

@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    if call.data == 'b1':
        bot.send_message(call.message.chat.id, 'Введите адрес сайта')
        bot.register_next_step_handler(call.message, get_url)
    elif call.data == 'b2':
        bot.send_message(call.message.chat.id, 'Введите текст')
        bot.register_next_step_handler(call.message, get_text)
    elif call.data == 'b3':
        sendkeyboard(1, bot.send_message(call.message.chat.id, 'Калькулятор'))
    elif call.data == 'b8':
        sendkeyboard(0, bot.send_message(call.message.chat.id, 'Back'))
    else:
        bot.send_message(call.message.chat.id, 'Введите первое число')
        bot.register_next_step_handler(call.message, get_schet1, call.data)


def get_url(message):
    if validators.url(message.text):
        request = requests.get(message.text)
        if request.status_code == 200:
            bot.send_message(message.chat.id, 'Сайт доступен')
        else:
            bot.send_message(message.chat.id, 'Ошибка '+str(request.status_code))
    else:
        bot.send_message(message.chat.id, 'Введён некорректный url')
    sendkeyboard(0, message)

def get_text(message): #Статистика, количество уникальных слов, самые популярные слова, кроме союзов и предлогов, количество предложений
    text = message.text
    splitedmessage = text.split()
    splitedmessagesentences = text.replace('!','.').replace('?','.').split('.')
    unicwords = set(list(map(lambda a: ''.join(filter(str.isalpha, a)),splitedmessage)))
    unicwvount = 0
    slovarik = {}
    sentcount = len(splitedmessagesentences)
    if text[len(text)-1]=='.' or text[len(text)-1]=='!' or text[len(text)-1]=='?':
        sentcount=sentcount-1
    for i in unicwords:
        if len(i)>2:
            unicwvount+=1
            slovarik[i]=(list(map(lambda a: ''.join(filter(str.isalpha, a)),splitedmessage))).count(i)
    answer = 'Кол-во уникальных слов - '+str(unicwvount)+', кол-во предложений - '+str(sentcount)+'\nПопулярность слов:\n'
    sortslov = dict(sorted(slovarik.items(), key=lambda i: i[1], reverse=True))
    for i in sortslov.items():
        answer+=i[0]+' - '+str(i[1])+'\n'
    bot.send_message(message.chat.id, answer)
    sendkeyboard(0, message)

def get_schet1(message, arg0):
    try:
        arg1 = int(message.text)
        bot.send_message(message.chat.id, 'Введите второе число')
        bot.register_next_step_handler(message, get_schet2, arg0, arg1)
    except:
        bot.send_message(message.chat.id, 'Ошибка!')
        sendkeyboard(1, message)

def get_schet2(message, arg0, arg1):
    try:
        arg2 = int(message.text)
        if arg0=='b4':
            arg2 = arg1*arg2
        elif arg0=='b5':
            arg2 = arg1/arg2
        elif arg0=='b6':
            arg2 = arg1+arg2
        elif arg0=='b7':
            arg2 = arg1-arg2
        bot.send_message(message.chat.id, str(arg2))
    except:
        bot.send_message(message.chat.id, 'Ошибка!')
    sendkeyboard(1, message)

bot.polling(none_stop=True, interval=0)